# Anki deck generator
## Based on the `genanki` library
Generalized anki deck generator. It uses the .pkl (pickle) files in the root directory to generate multiple nested decks for Anki, by language and categorie.
- File format (Pandas dataframe as Python Pickle): `<language-name>.pkl` or `<language-name>-<word-categorie>.pkl`

In [5]:
# Imports
import genanki
import pandas as pd
import glob

In [6]:
# Define general genanki model
my_model = genanki.Model(
  202208030953, # 2022-08-03- 09:53
  'Nota basica con audio',
  fields=[
    {'name': 'Front'},
    {'name': 'Audio'},
    {'name': 'Back'},
  ],
  css='''
  .card {
    font-family: arial;
    font-size: 20px;
    text-align: center;
    color: black;
    background-color: white;
  }  
  ''',
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Front}}<br>{{Audio}}',
      'afmt': '{{FrontSide}}<hr id=answer>{{Back}}',
    },
  ])

In [7]:
def load_pkl_files():
    dataframes = {}
    for file in glob.glob('*.pkl'):
        print(f'Filename: {file}') 
        filename_divided = file.replace('.pkl', '').split('-')
        temp_df = pd.read_pickle(file)
        elements = len(filename_divided)
        if elements == 1:
            language = filename_divided[0]
        elif elements == 2:
            language, categorie = filename_divided
            temp_df['categoria'] = categorie
        else:
            raise ValueError('Wrong filename. <language>-<categorie>.pkl for non classified files or <language>.pkl for classified files')
        dataframes[language] = temp_df
    return dataframes
        

In [8]:
def correct_categorie(cat: str) -> str:
    if cat == 'nombre':
        return 'Sustantivo'
    elif cat == 'nombre propio':
        return 'Sustantivo propio'
    else:
        return cat

In [9]:
def generate_decks(language, df) -> list:
    print(language)
    decks = []
    categories = df.categoria.unique()
    for cat in categories:
        df_by_cat = df[df.categoria == cat]
        deck_name = f'{language.capitalize()}::{correct_categorie(cat).capitalize()}'
        my_deck = genanki.Deck(hash(deck_name), deck_name)
        for _, row in df_by_cat.iterrows():
            # print(row["Name"], row["Age"])
            card = genanki.Note(model=my_model, fields=[row['palabra'], '', row['traduccion']])
            my_deck.add_note(card)
        decks.append(my_deck)
    return decks

In [10]:
def package_and_save(filename, deck):
    genanki.Package(deck).write_to_file(f'{filename}.apkg')

In [11]:
dataframes = load_pkl_files()

Filename: aleman-sustantivo.pkl
Filename: ruso.pkl


In [12]:
for language, df in dataframes.items():
    decks  = generate_decks(language, df)
    package_and_save(language, decks)

aleman
ruso
